In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [4]:
text="""박교수가 NLP 강의를 한다
학생들이 NLP 강의를 듣는다
학생들이 NLP 강의를 이해 못한다"""

In [5]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text]) #fit_on_texts: 문자 데이터를 입력받아서 리스트의 형태로 변환

In [6]:
tokenizer.word_index #word_index: 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환

{'nlp': 1,
 '강의를': 2,
 '학생들이': 3,
 '박교수가': 4,
 '한다': 5,
 '듣는다': 6,
 '이해': 7,
 '못한다': 8}

In [7]:
vocab_size=len(tokenizer.word_index)+1
print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 9


In [8]:
text.split('\n')

['박교수가 NLP 강의를 한다', '학생들이 NLP 강의를 듣는다', '학생들이 NLP 강의를 이해 못한다']

In [9]:
tokenizer.texts_to_sequences([text.split('\n')[0]])

[[4, 1, 2, 5]]

In [10]:
tokenizer.texts_to_sequences([text.split('\n')[0]])[0] #texts_to_sequences: text 문장을 숫자로 이루어진 리스트로 만든다. 

[4, 1, 2, 5]

In [11]:
encoded_results=list()
for line in text.split('\n'): #\n, 줄바꿈 기준으로 문장 토큰화
    encoded=tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1,len(encoded)):
        sequence=encoded[:i+1]
        encoded_results.append(sequence)

print("학습에 사용할 샘플의 개수: %d" % len(encoded_results))

학습에 사용할 샘플의 개수: 10


In [12]:
encoded_results

[[4, 1],
 [4, 1, 2],
 [4, 1, 2, 5],
 [3, 1],
 [3, 1, 2],
 [3, 1, 2, 6],
 [3, 1],
 [3, 1, 2],
 [3, 1, 2, 7],
 [3, 1, 2, 7, 8]]

In [13]:
max_len=max(len(i) for i in encoded_results)

In [16]:
pad_sequences=pad_sequences(encoded_results, maxlen=max_len, padding='pre') #pre:0을 앞에다가

In [17]:
pad_sequences

array([[0, 0, 0, 4, 1],
       [0, 0, 4, 1, 2],
       [0, 4, 1, 2, 5],
       [0, 0, 0, 3, 1],
       [0, 0, 3, 1, 2],
       [0, 3, 1, 2, 6],
       [0, 0, 0, 3, 1],
       [0, 0, 3, 1, 2],
       [0, 3, 1, 2, 7],
       [3, 1, 2, 7, 8]])

In [18]:
pad_sequences=np.array(pad_sequences)
X=pad_sequences[:,:-1] 
Y=pad_sequences[:,-1] #마지막 컬럼만

In [19]:
X

array([[0, 0, 0, 4],
       [0, 0, 4, 1],
       [0, 4, 1, 2],
       [0, 0, 0, 3],
       [0, 0, 3, 1],
       [0, 3, 1, 2],
       [0, 0, 0, 3],
       [0, 0, 3, 1],
       [0, 3, 1, 2],
       [3, 1, 2, 7]])

In [20]:
#to_categorical: 원-핫 인코딩을 해주는 함수
y_cate=to_categorical(Y,num_classes=vocab_size) #9개의 카테고리

In [21]:
Y

array([1, 2, 5, 1, 2, 6, 1, 2, 7, 8])

In [22]:
y_cate

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [23]:
embedding_layer=Embedding(vocab_size, 4, trainable=True, input_length=max_len) #4개짜리 크기의 벡터

In [24]:
embedding_dim=4
hidden_units=10

RNN=Sequential()
RNN.add(embedding_layer)

RNN.add(SimpleRNN(units=hidden_units, activation='relu')) #units:히든 레이어의 히든 노드 개수
RNN.add(Dense(vocab_size, activation='softmax')) #9개의 단어에 대한 확률값을 뱉어줌

In [25]:
print(RNN.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 4)              36        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 10)                150       
_________________________________________________________________
dense (Dense)                (None, 9)                 99        
Total params: 285
Trainable params: 285
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
RNN.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #멀티 클래스 문제기이 때문에 categorical_crossentropy 사용

In [27]:
RNN.fit(X,y_cate, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 986ms/step - loss: 2.2027 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - loss: 2.1990 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 2.1960 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 2.1931 - accuracy: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 2.1902 - accuracy: 0.0000e+00
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 2.1873 - accuracy: 0.0000e+00
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 2.1844 - accuracy: 0.0000e+00
Epoch 8/200
1/1 [==============================] - 0s 3ms/step - loss: 2.1812 - accuracy: 0.0000e+00
Epoch 9/200
1/1 [==============================] - 0s 3ms/step - loss: 2.1778 - accuracy: 0.2000
Epoch 10/200
1/1 [==============================] - 0s 4ms/step - loss: 2.1743 - accuracy: 0.

1/1 [==============================] - 0s 2ms/step - loss: 1.8697 - accuracy: 0.7000
Epoch 79/200
1/1 [==============================] - 0s 3ms/step - loss: 1.8621 - accuracy: 0.7000
Epoch 80/200
1/1 [==============================] - 0s 2ms/step - loss: 1.8543 - accuracy: 0.7000
Epoch 81/200
1/1 [==============================] - 0s 2ms/step - loss: 1.8465 - accuracy: 0.7000
Epoch 82/200
1/1 [==============================] - 0s 3ms/step - loss: 1.8385 - accuracy: 0.7000
Epoch 83/200
1/1 [==============================] - 0s 3ms/step - loss: 1.8303 - accuracy: 0.7000
Epoch 84/200
1/1 [==============================] - 0s 2ms/step - loss: 1.8221 - accuracy: 0.7000
Epoch 85/200
1/1 [==============================] - 0s 2ms/step - loss: 1.8137 - accuracy: 0.7000
Epoch 86/200
1/1 [==============================] - 0s 2ms/step - loss: 1.8052 - accuracy: 0.7000
Epoch 87/200
1/1 [==============================] - 0s 3ms/step - loss: 1.7965 - accuracy: 0.7000
Epoch 88/200
1/1 [===============

Epoch 161/200
1/1 [==============================] - 0s 2ms/step - loss: 0.9109 - accuracy: 0.8000
Epoch 162/200
1/1 [==============================] - 0s 2ms/step - loss: 0.9003 - accuracy: 0.8000
Epoch 163/200
1/1 [==============================] - 0s 3ms/step - loss: 0.8899 - accuracy: 0.8000
Epoch 164/200
1/1 [==============================] - 0s 3ms/step - loss: 0.8796 - accuracy: 0.8000
Epoch 165/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8695 - accuracy: 0.8000
Epoch 166/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8595 - accuracy: 0.8000
Epoch 167/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8496 - accuracy: 0.8000
Epoch 168/200
1/1 [==============================] - 0s 3ms/step - loss: 0.8399 - accuracy: 0.8000
Epoch 169/200
1/1 [==============================] - 0s 2ms/step - loss: 0.8303 - accuracy: 0.8000
Epoch 170/200
1/1 [==============================] - 0s 3ms/step - loss: 0.8209 - accuracy: 0.8000
Epoch 171/

In [14]:
test_encode=tokenizer.texts_to_sequences(['학생들이 NLP 강의를'])[0]

In [15]:
pad_test_encode=pad_sequences([test_encode], maxlen=4, padding='pre')

In [28]:
pad_test_encode

array([[0, 3, 1, 2]])

In [29]:
pred_prob=RNN.predict(pad_test_encode)

In [30]:
tokenizer.word_index

{'nlp': 1,
 '강의를': 2,
 '학생들이': 3,
 '박교수가': 4,
 '한다': 5,
 '듣는다': 6,
 '이해': 7,
 '못한다': 8}

In [31]:
pred_prob

array([[0.00567153, 0.0097875 , 0.05740198, 0.00508699, 0.00042935,
        0.3087412 , 0.32722867, 0.14627023, 0.1393824 ]], dtype=float32)

In [32]:
pred_class=np.argmax(pred_prob, axis=1) #argmax:가장 큰 숫자의 해당하는 인덱스를 알려줌

In [33]:
pred_class

array([6], dtype=int64)